In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_absolute_error
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeRegressor
import numpy as np

In [ ]:
df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')
subsample_df = pd.read_csv('submission_sample.csv')
features_df = pd.read_csv('features.csv')

Для обучения модели возьмем train.csv и разделим его в соотношении ~2:1. Используется регрессия с деревом решений

In [ ]:
X, Y = df.loc[:2000, 'lat':'lon'], df.loc[:2000, 'score']

In [ ]:
clf = DecisionTreeRegressor(random_state=200)
clf.fit(X, Y)
my_predict = clf.predict(df.loc[2000:, 'lat':'lon'])
mean_absolute_error(df.loc[2000:, 'score'], my_predict)

0.04635089892549479

Данные из features.csv подвергнем кластеризации, т.е. разобъем их на классы. Полученнную модель используем для предсказания классов для данных из train.csv и запишем их как дополнительное поле фичей

In [ ]:
imp = SimpleImputer(missing_values=np.nan, strategy='mean')
new_feature_df = imp.fit_transform(features_df)

kmeans = KMeans(n_clusters=5, random_state=5, n_init="auto").fit(new_feature_df)
data = features_df.loc[:, :'lon']
imp = SimpleImputer(missing_values=np.nan, strategy='mean')
new_df = imp.fit_transform(data)

neigh = KNeighborsClassifier(n_neighbors=5)
neigh.fit(new_df, kmeans.labels_)
lbls_pred = neigh.predict(df.loc[:,'lat':'lon'])

new_x = df.loc[:, 'lat':'lon'].assign(e =lbls_pred)
Y = df.loc[:, 'score']

clf = DecisionTreeRegressor(random_state=25)
clf.fit(new_x[:2000], Y[:2000])
my_predict = clf.predict(new_x.loc[2000:])

mean_absolute_error(Y[2000:], my_predict)

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:432: UserWarning: X has feature names, but KNeighborsClassifier was fitted without feature names
  warnings.warn(


0.044545008959340215

In [ ]:
delta = 0.04635089892549479 -0.044545008959340215
print("Улучшение ошибки составило: ", delta)
print("Улучшение ошибки составило (в процентах): ", round(delta/0.04635089892549479*100, 3), "%")

Улучшение ошибки составило:  0.001805889966154578
Улучшение ошибки составило (в процентах):  3.896 %


***Далее идет код непосредственно предсказания score для test.csv***

In [83]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_absolute_error
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeRegressor
import numpy as np

In [84]:
df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')
features_df = pd.read_csv('features.csv')

In [85]:
imp = SimpleImputer(missing_values=np.nan, strategy='mean')
new_feature_df = imp.fit_transform(features_df)

kmeans = KMeans(n_clusters=5, random_state=5, n_init="auto").fit(new_feature_df)

data = features_df.loc[:, :'lon']
imp = SimpleImputer(missing_values=np.nan, strategy='mean')
new_df = imp.fit_transform(data)

neigh = KNeighborsClassifier(n_neighbors=5)
neigh.fit(new_df, kmeans.labels_)
lbls_pred = neigh.predict(df.loc[:,'lat':'lon'])

X = df.loc[:, 'lat':'lon'].assign(e =lbls_pred)
Y = df.loc[:, 'score']

clf = DecisionTreeRegressor(random_state=25)
clf.fit(X, Y)


lest_labels_predict = neigh.predict(test_df.loc[:,'lat':'lon'])
test_x = test_df.loc[:, 'lat':'lon'].assign(e =lest_labels_predict)


my_predict = clf.predict(test_x.loc[:, 'lat':])

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:432: UserWarning: X has feature names, but KNeighborsClassifier was fitted without feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:432: UserWarning: X has feature names, but KNeighborsClassifier was fitted without feature names
  warnings.warn(


In [86]:
answer = test_df.drop('lat', axis = 1)
answer = answer.drop('lon', axis = 1)
answer['score'] = pd.Series(my_predict)
answer.to_csv("submission.csv")